In [1]:
using Oscar
using Combinatorics
pm = Polymake

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.11.2 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2022 by The OSCAR Development Team


Polymake

In [63]:
include("matroid_realization.jl");
include("reduceIdeals.jl")
include("d3n10_special_matroids.jl")

d3n10 = vec(readlines("d3n10.dat"));


#remove monomial factors
function clean(f,R)
    
    if !(f == 0)
    
        if length(f) == 1
            
            return R(coeff(f,1))
            
        else
            
            fFactors = factor(f)
    
            FactorsDict = Dict(fFactors)
    
            cleanf = prod([k^(FactorsDict[k]) for k in keys(FactorsDict) if (length(k)>1||is_constant(k))])
        
            return cleanf
        
        end
        
    else
        
        return f
        
    end
    
end
    

function matroid_to_reduced_expression(Q, F,k)
    
    charts = [c for c in circuits(Q) if length(c) == rank(Q)+1]
    A = charts[1]
    RQ = matroid_realization_space(Q, A, F)
    R = parent(RQ[1][1])
    Sgens = [s for s in RQ[2] if length(s) <= k]#new 13.1.2023
    I = reduce_ideal_full(RQ[1], Sgens, R, gens(R), false)
    
    
 
    if I isa String
        
        return I
        
     #I[1] = ideal generators, I[2] = subgroup generators   
    else
        
       Iclean = unique!([clean(f,R) for f in I[1]])
    
            
       return (Iclean, I[2])
        
       # return(I[1],I[2])
        
    end
end




matroid_to_reduced_expression (generic function with 1 method)

In [73]:
Mz = matroid_from_revlex_basis_encoding(d3n10[4917], 3, 10)
I = matroid_to_reduced_expression(Mz, ZZ,2)

int_gens(I[1])

true

In [3]:
d3n10

5249-element Vector{String}:
 "*******************************" ⋯ 58 bytes ⋯ "*******************************"
 "0******************************" ⋯ 58 bytes ⋯ "*******************************"
 "0******************0***********" ⋯ 58 bytes ⋯ "*******************************"
 "0******************0***********" ⋯ 58 bytes ⋯ "*******************************"
 "0******0***********************" ⋯ 58 bytes ⋯ "*******************************"
 "0******0***********************" ⋯ 58 bytes ⋯ "*******************************"
 "0******0***********************" ⋯ 58 bytes ⋯ "****************************0**"
 "0******0***********************" ⋯ 58 bytes ⋯ "*******************************"
 "0******0***********************" ⋯ 58 bytes ⋯ "******************************0"
 "0******0***********************" ⋯ 58 bytes ⋯ "*******************************"
 "0******0***********************" ⋯ 58 bytes ⋯ "*********************0*********"
 "0******0***********************" ⋯ 58 bytes ⋯ "********************

In [92]:
has_property #matroids with "3 lines property"

151-element Vector{Any}:
  324
  423
  428
  450
  630
  635
  798
  884
  894
  903
  904
  906
  908
    ⋮
 4810
 4822
 4825
 4829
 4839
 4841
 4903
 4910
 4916
 4917
 4919
 5085

In [4]:
#for an element n, counts how many lines its in
function in_lines(n,L)
    xn = 0
    for z in 1:length(L)
        if n in L[z]
           xn += 1 
        end
    end
    return xn
end

in_lines (generic function with 1 method)

In [5]:
#find simple matroids such that each element is contained in at least 3 lines
#this could be a function that depends on M and k, where k is the number of "lines"
has_property = []
for z in 1:length(d3n10)
    Mz = matroid_from_revlex_basis_encoding(d3n10[z], 3, 10)
    L = cyclic_flats(Mz,2)
    xm = 0
    for t in 1:10
        if in_lines(t,L)>2
            xm +=1
        end
    end
    #print(z,"\n", xm ,"\n",find_lines(z),"\n")
    if xm == 10
        push!(has_property,z)
    end
end

In [91]:
#realized i should check that everything is connected
for z in has_property
    Mz = matroid_from_revlex_basis_encoding(d3n10[z], 3, 10)
    if !(n_connected_components(Mz) == 1)
        print(z,"\n")
        
    end
    
end

In [71]:
function f_needs_more(I)
    
    cc = []
    
    for f in I
        
        g = [x for x in vars(f) if degree(f,x) == 1]
        
        if length(g)>0
            
            push!(cc,f)
            
        end
        
    end
        
        if length(cc)>0
            
            return "more work"
            
        else
            
            return "fully simplified"
            
        end
        
    end
        
            


f_needs_more (generic function with 1 method)

In [72]:
function int_gens(I)
    
    ints = [x for x in I if is_constant(x)]
    
    if length(ints)>1
        
        return true
        
    else
        
        return false
        
    end
    
end

int_gens (generic function with 1 method)

In [75]:
#classification

not_realizable = []

int_generator = []#algebraic closure of integral generator not equal to 1

need_more_not_principle = []#ideal not fullysimplified, not principle
need_more_principle = [] #ideals that simplifies to something principle, but can be further reduced

simplified_not_principle = []#ideal simplifies,al not principle
simplified_principle_n_reducible = [] #ideal simplifies to something principle
simplified_principle_reducible = []#ideal fully simplifies, principally generated in one variable

zero_ideal = []#ideal fully simplifies to 0

for z in has_property
    
    #println(z,"\n\n")
  
    Mz = matroid_from_revlex_basis_encoding(d3n10[z], 3, 10)
    I = matroid_to_reduced_expression(Mz, ZZ,2)
    
    #println("ch1")
    if I[1] == [0] #reduces to 0
        
        push!(zero_ideal,z)
    
    elseif (1 in I[1] || -1 in I[1] || I == "Not Realizable")
        
    #println("ch2")
        
        push!(not_realizable,z)
        
    elseif int_gens(I[1])
        
               
    #println("ch3")
        
        push!(int_generator,z)
        
    elseif f_needs_more(I[1]) == "more work"#not fully reduced
        
               
    #println("ch4")
        
        if length(I[1]) == 2
            
            push!(need_more_principle,z)
            
        else
        
            push!(need_more_not_principle,z)
            
        end
        
    elseif f_needs_more(I[1]) == "fully simplified"#fully reduced
        
               
   # println("ch5")
        
        if length(I[1]) == 2
            
            if length(vars(I[1][2])) == 1
                
                push!(simplified_principle_reducible,z)
                
            else
                
                push!(simplified_principle_n_reducible,z)
            end
            
        else
        
            push!(simplified_not_principle,z)
            
        end
        
    end
end


In [76]:
C = [not_realizable,
int_generator,

need_more_not_principle,
need_more_principle,

simplified_not_principle,
simplified_principle_n_reducible,
simplified_principle_reducible,

zero_ideal]

8-element Vector{Vector{Any}}:
 [1072, 1546, 2671, 2742, 3123, 3599, 3658, 3666, 3725, 4825]
 [1607, 3648, 3802, 3840, 4493, 4789, 4800, 4808, 4809, 4810, 4839, 4841, 4910, 4917, 4919]
 [635, 1412, 2669, 3592]
 [630, 1080, 1417, 1438, 1544, 1558, 2460, 2743, 3156, 3417, 3419, 3426, 3626, 3634, 3770, 3801, 4822]
 [1521, 1522, 1543, 1556, 1593, 1602, 3588, 3645, 3647, 4568]
 [324, 423, 428, 450, 884, 894, 903, 904, 906, 908  …  1457, 2934, 3121, 3124, 3154, 3157, 3234, 3235, 3349, 3359]
 [798, 965, 995, 1524, 1594, 1596, 1598, 1674, 1675, 2740  …  3809, 4276, 4490, 4557, 4570, 4571, 4758, 4761, 4903, 5085]
 [1082, 1525, 1672, 2459, 2739, 2786, 2955, 2958, 3352, 3403, 3726, 3771, 3800, 3811, 3812, 3839, 4279, 4784, 4829, 4916]

In [77]:
for c in C
    print(length(c),"\n")
end
lc = [length(c) for c in C]
sum(lc)

10
15
4
17
10
45
30
20


151

In [88]:
for z in C[4]
    
     Mz = matroid_from_revlex_basis_encoding(d3n10[z], 3, 10)
     I = matroid_to_reduced_expression(Mz, ZZ,2)
    
    print("Matroid : ",z,"\n\n")
    
    if I isa String

        print(I,"\n\n")
        
    else
        
    for j in 1:length(I[1])
        print(j," : ", factor(I[1][j]),"\n\n")
    end
    end
end

Matroid : 630

1 : 0

2 : 1 * (x_{1, 6} - 1) * (2*x_{2, 5} - 1)

Matroid : 1080

1 : 0

2 : 1 * (x_{1, 4}*x_{2, 5} - x_{1, 4}*x_{2, 6} - x_{2, 5}*x_{2, 6} + 2*x_{2, 6} - 1) * (x_{1, 4} - x_{2, 5})

Matroid : 1417

1 : 0

2 : 1 * (x_{1, 6}^2 + x_{1, 6}*x_{2, 6} - x_{1, 6} - 2*x_{2, 6} + 1)

Matroid : 1438

1 : 0

2 : 1 * (x_{1, 5}^2 + 2*x_{1, 5}*x_{2, 6} - 2*x_{1, 5} - x_{2, 6})

Matroid : 1544

1 : 0

2 : 1 * (x_{2, 6} - 1)

Matroid : 1558

1 : 0

2 : 1 * (2*x_{1, 6}*x_{2, 6} - x_{1, 6} - x_{2, 6}^2)

Matroid : 2460

1 : 0

2 : 1 * 2 * (x_{1, 6} - 1)

Matroid : 2743

1 : 0

2 : 1 * 2 * (x_{1, 6} - 1)

Matroid : 3156

1 : 0

2 : 1 * (x_{2, 4}*x_{2, 6} - x_{2, 6}^2 + 1)

Matroid : 3417

1 : 0

2 : 1 * (x_{1, 6}^2*x_{2, 6} - x_{1, 6}^2 - x_{1, 6}*x_{2, 6} + x_{2, 6})

Matroid : 3419

1 : 0

2 : 1 * (x_{1, 5} - 2) * (x_{1, 5}*x_{2, 6} - 1)

Matroid : 3426

1 : 0

2 : 1 * (x_{1, 6}^2*x_{2, 6} - 2*x_{1, 6}^2 + x_{2, 6})

Matroid : 3626

1 : 0

2 : 1 * (x_{2, 5} + 2)

Matroid : 3634

1 : 0

2